In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import quad

In [ ]:
def findxi(beta,emit,gamma,intensity,sigs,r0,xing):
    sigx = np.sqrt(beta*emit/gamma)
    redFact = 1.0/np.sqrt(1.0+(sigs*np.tan(xing/2)/sigx)**2)
    xi = intensity*r0*redFact/(2.0*np.pi*emit)
    return xi

In [9]:
def getLebedev(xi,delta,dGain,deltaBPM,dmu):
    growths = np.zeros_like(dGain)
                
    for i in range(len(dGain)):
        #tune shift decreasing and noise increasing
        #contribution of adt increasing with intensity ^2 and xi change due to nb change
        #http://lhcinstability.web.cern.ch/lhcinstability/Presentations/2018-05-25-lhcim_oeftiger_report.pdf
        #http://lhcinstability.web.cern.ch/lhcinstability/
        #https://indico.cern.ch/event/754955/contributions/3128565/attachments/1709927/2756334/2018.08.22-29_LHCIM_Weeksummary.pdf
        relGrowth0 = (delta**2+(dGain[i]*deltaBPM)**2)/2
        f = lambda x : (4*np.pi**2*(1-dGain[i]/2)**2*x**2)*np.exp(-x**2/(2.0*dmu**2))/((4*np.pi**2*(1-dGain[i]/2)*x**2+(dGain[i]/2)**2)*np.sqrt(2*np.pi)*dmu)
        integral = quad(f,-10*dmu,10*dmu)
        growths[i] = relGrowth0*integral[0]
    return growths

In [11]:
#deltaBPMnow=deltaBPM*(1.2e11/Inow)^k for k=1/2?
def getNoiseGrowth(energy=None, sigs=None, intensity=None, emit=None, xing=None, beta=None, delta=None, deltaBPM_SB=None,sigmaElastic_m2=None):
    gamma = energy/0.938  
    #dmu for non-colliding
    #dmu_nonHILUMI = 9.3e-5 #varies linearly with 2.5emit, 570A  and 1/7TeV, so.... https://cds.cern.ch/record/2301292/files/CERN-ACC-NOTE-2018-0002.pdf
    #print emit
    dmu_non = dmu_non_factor*emit #LHC estimation
    #k=1./2. #this value would be more or less the same for both planes and beams
    #k=1.0
    deltaBPMnow=deltaBPM_SB*(1.2e11/intensity)**k #for k=1/2? 1.2 is the reference intensity for which we have optimized, not the 2e11 of the MD
    #energy = 6500.0
    #sigs = 0.0824 #for 1.1ns
    #intensity = 1.08E11 #for 2018 intens. at SB
    #emit = 1.9E-6 #for 2018 emit at SB     # for the MD: 2.3E-6
    #xing = 2*160e-6
    #beta = 0.3 #we use the betastar that is 30 cm , since it is used for the emit. in the xi calculation
    '''
    sigx = np.sqrt(beta*emit/gamma)
    ## approx formula for the total tune shift (2IPs) with (alternating) crossing angles
    redFact = 1.0/np.sqrt(1.0+(sigs*np.tan(xing/2)/sigx)**2)
    xi = intensity*r0*redFact/(2.0*np.pi*emit)
    '''
    #print beta,emit,gamma,intensity,sigs,r0,xing
    #if sigmaElastic_m2 != 0: #colliding 
    #print beta,emit,gamma,intensity,sigs,r0,xing   
    xi=findxi(beta,emit,gamma,intensity,sigs,r0,xing)
    
    #xi_vb1=findxi(beta,emit,gamma,intensity,sigs,r0,xing)
    #xi_hb2=findxi(beta,emit,gamma,intensity,sigs,r0,xing)
    #xi_vb2=findxi(beta,emit,gamma,intensity,sigs,r0,xing)
    #else:                    #non-colliding 
    #    xi=0
    #    xi_hb1=0
    #    xi_vb1=0
    #    xi_hb2=0
    #    xi_vb2=0
        
    #xi = 0.02
    #print sigmaElastic_m2
    if sigmaElastic_m2 != 0: #colliding 
        #print 'xi',xi
        dmu = 0.168*xi #octupoles?
        #print 'dmu col:',dmu
    else:                    #non-colliding 
        dmu = dmu_non
        #print 'dmu non-col:',dmu
    #print deltaBPMnow    
    growthsFT= getLebedev(xi,delta,gains,deltaBPMnow,dmu)*frev*3600*100 # rel. emit growth in %/h
    #growthsFT_vb1 = getLebedev(xi_vb1,delta_vb1,gains,deltaBPM_SB_vb1)*frev*3600*100 # rel. emit growth in %/h
    #growthsFT_hb2 = getLebedev(xi_hb2,delta_hb2,gains,deltaBPM_SB_hb2)*frev*3600*100 # rel. emit growth in %/h
    #growthsFT_vb2 = getLebedev(xi_vb2,delta_vb2,gains,deltaBPM_SB_vb2)*frev*3600*100 # rel. emit growth in %/h
    
    getGrowth=growthsFT[gains==theGainFT]
    #getGrowth_vb1=growthsFT_vb1[gains==theGainFT]
    #getGrowth_hb2=growthsFT_hb2[gains==theGainFT]
    #getGrowth_vb2=growthsFT_vb2[gains==theGainFT]
    
    #print getGrowth_hb1, getGrowth_vb1
    #print 'HB1 noiseGrowth at SB:',float(emit*getGrowth_hb1/100)*1e6,'um/h'
    #print 'VB1 noiseGrowth at SB:',float(emit*getGrowth_vb1/100)*1e6,'um/h'
    #print 'HB2 noiseGrowth at SB:',float(emit*getGrowth_hb2/100)*1e6,'um/h'
    #print 'VB2 noiseGrowth at SB:',float(emit*getGrowth_vb2/100)*1e6,'um/h'
    
    return getGrowth,deltaBPMnow,xi